# Module 6: Automating Scientific Discovery
---

Welcome to the practical session of [Data-Driven Life Sciences course module 6](https://ddls.aicell.io/course/ddls-2024/module-6/), created by Gabriel Reder. And, Professor Wei Ouyang, teaching assistants Songtao Cheng, and Nils Mechtel made some modifications.

## Introduction
This notebook will guide exploration of automating the experimental side of scientific discovery in the laboratory. The aim is to encourage thinking about everyday manual tasks in experiments that can be aided through the use of LLM models and AI systems boosting the power of laboratory robotics.

**Authors**: Gabriel Reder (gk@reder.io)

# TODO: more introduction

We will use concepts that were discussed in the lecture, such as lab automation and hypha to connect your mcp tool to Gemini.

### Important Note for This Lab Notebook:

- **🌞 Tasks Introduction:** Sections marked with a 🌞 symbol introduce an exercise or question. Please read these sections carefully to understand the concepts and tasks involved.

- **⭐ Your Answer Here:** Cells marked with a ⭐ symbol indicate where you need to write your answer. Please provide your code or answer there.

---

## Setup

### Save the notebook to your Module 6 folder in Google Drive

Before you start, copy this notebook to your Google Drive:

`File` -> `Save a copy in Drive`

(You can close the other tab with the original notebook.)

Next, move the copied notebook to your Module 6 folder in Google Drive:

`File` -> `Move` -> Go up to `My Drive` -> Select your `DDLS-Course` folder -> Create a new folder named `Module6` -> `Select folder`

### Runtime

You can run this notebook with a **CPU** runtime. A GPU runtime is not required because of the small model size.

### Mount Google Drive Folder

Mounting your Google Drive folder allows your Colab notebook to access files stored in your Google Drive. This is useful for loading datasets, saving results, or accessing other files you need for your work.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

If you copied and saved the notebook correctly, you should now see this notebook in your Google Drive under `/content/drive/MyDrive/DDLS-Course/Module6/`.

In [ ]:
!ls -lh /content/drive/MyDrive/DDLS-Course/Module6/

## Installing dependencies
We'll need the `opentrons` python package to simulate running protocols, so we'll install that now.

In [ ]:
!pip install opentrons mcp pydantic hypha_rpc

### Working with Gemini CLI in Google Colab

Open the terminal in Google Colab and run the following commands to install Gemini CLI:

```bash
curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.40.3/install.sh | bash
source /root/.bashrc
nvm install 21
nvm use 21
npm install -g @google/gemini-cli
```

**Start Gemini CLI**

To start a Gemini chat session, first change your working directory to your Module 5 folder in Google Drive:

```bash
cd /content/drive/MyDrive/DDLS-Course/Module6/
```

and then run the `gemini` command:

```bash
gemini
```

**Save Your Gemini Chat History**

Remember to regularly save your Gemini chat history to avoid losing records of your work. In the Gemini CLI, run:
```
/chat save computer-lab-6
```

**Copy the Checkpoint File**

Ask Gemini to copy the checkpoint file from the temporary Gemini directory to your Google Drive folder while you are still in the Gemini CLI session:
```
Run this command: `cp /root/.gemini/tmp/*/checkpoint-computer-lab-6.json .`
```

You can do so after every small or large step of your work. This will prevent the loss of your chat history if runtime in Google Colab disconnects and gets reset.

If you have already closed the Gemini CLI session, run this command to copy the checkpoint file to your Google Drive folder:
```bash
cd /content/drive/MyDrive/DDLS-Course/Module6/
cp /root/.gemini/tmp/*/checkpoint-computer-lab-6.json .
```

**Load Your Gemini Chat History**

To load your previously saved Gemini chat history, run the following command in the Gemini CLI:
```
/chat load computer-lab-6
```

**General tips when working on large data files**

Don’t let Gemini read the entire files. Instead use preview commands like `head` or `tail`. Also avoid printing details and always keep the outputs concise.

It can help to add instructions like this to your GEMINI.md file.

---

# Creating robotic lab protocols

## Opentron liquid handlers
For this exercise, we'll suppose you are running a lab equipped with an Opentron flex liquid handling robot:

<figure>
<center>
<img src="https://cdn11.bigcommerce.com/s-zmwgbev7rb/products/255/images/814/Flex-6__84033.1719413544.386.513.png?c=1" width="350">
</figure>

If you'd like to see the robot in action, take a look at this marketing [video](https://www.youtube.com/watch?v=d6ln-0LeT8A)

## A sample Opentron protocol

One thing that's nice about Opentrons is that they have a Python API, meaning we can write scripts to execute protocols directly on the robot. Take a look at the sample protocol below. Can you tell what's going on?

In [ ]:
from opentrons.simulate import get_protocol_api


def sample_protocol():
    # Create a ProtocolContext for the Flex robot
    protocol = get_protocol_api(
        version="2.19", robot_type="Flex"  # API version 2.19  # Flex robot
    )
    print("Protocol API version:", protocol.api_version)

    # Load the labware (plate, tiprack, trash)
    plate = protocol.load_labware(
        load_name="corning_96_wellplate_360ul_flat", location="D1"
    )
    print("Loaded labware:", plate)

    tiprack_1 = protocol.load_labware(
        load_name="opentrons_flex_96_tiprack_200ul", location="D2"
    )
    print("Loaded labware:", tiprack_1)

    trash = protocol.load_trash_bin(location="D3")
    print("Loaded labware:", trash)

    # Load the instrument (pipette)
    pipette = protocol.load_instrument(
        instrument_name="flex_1channel_1000",
        mount="left",
        tip_racks=[tiprack_1],  # specify the tiprack(s) to pull tips from
    )
    print("Loaded instrument:", pipette)

    # Define the protocol steps
    print("Starting protocol...")

    pipette.pick_up_tip(location=tiprack_1)
    print("Picked up tip from A1")

    pipette.aspirate(volume=100, location=plate["A1"])
    print("Aspirated 100uL from A1")

    pipette.dispense(volume=100, location=plate["B1"])
    print("Dispensed 100uL to B1")

    pipette.drop_tip(location=trash)
    print("Dropped tip to trash")

    print("Protocol complete.")

## Simulating protocols

The opentron API allows us to simulate protocols, even if we don't have a robot. Let's try this out on our sample protocol. Make sure you have executed the cell above containing the sample protocol.

Now, execute the cell below to simulate the sample protocol and see what it does.

In [34]:
sample_protocol()

/Users/nilsm/.opentrons/robot_settings.json not found. Loading defaults
Belt calibration not found.


Protocol API version: 2.19
Loaded labware: Corning 96 Well Plate 360 µL Flat
Loaded labware: Opentrons Flex 96 Tip Rack 200 µL
Loaded labware: <opentrons.protocol_api.disposal_locations.TrashBin object at 0x137aac310>
Loaded instrument: Flex 1-Channel 1000 µL on left mount
Starting protocol...
Picked up tip from A1
Aspirated 100uL from A1
Dispensed 100uL to B1
Dropped tip to trash
Protocol complete.


🌞 <font color='orange'>**Exercises**:</font>

**Note** - Do these exercises manually, do *not* use your AI agent yet.

1. Watch the opentron protocol designer [video](https://support.opentrons.com/s/article/How-to-write-a-basic-protocol-in-Protocol-Designer) to familiarize yourself with the robot and the environment. We're going to use the Python API directly rather than the `Protocol Designer` GUI. After watching the video, what do you think the protocol above is doing? Write a step-by-step plain text protocol that corresponds to what you think the protocol does.

2. What do the `location` designations (e.g. `D1` and `D2`) in the script correspond to?

3. Now familiarize yourself with the opentron [API](https://docs.opentrons.com/v2/) and pay special attention to the [Hardware Modules](https://docs.opentrons.com/v2/new_modules.html#) and the [Labware Library](https://labware.opentrons.com/). What is the difference between `Hardware` and `Labware` in the context of the opentron API?

4. Find the API names for the following pieces of hardware/labware:
  - Thermocycler module
  - Magnetic block
  - GEB 96 Tip Rack 10 µL
  - Thermo Scientific Nunc 96 Well Plate 1300 µL



---
⭐ Double click to write down your answers here


```
Answer:

1.

2.

3.

4.

```
---

🌞 <font color='orange'>**Exercises**:</font>

You've been given a 96-well plate where column 1 contains 300$\mu$L of sample in each well of the column. You want to perform a serial dilution at 1:10 dilution factors across each column. So column 1 will contain the sample, column 2 will contain the sample diluted at 1:10, column 3 will be another 1:10 dilution (so 1:100 from the original sample) etc. You have the following working parameters for the protocol:

  - You are diluting the sample into water. You have a reservoir of water available to you on the opentron.
  - You want the final dilution column to be a 1:1000000 dilution from the original sample.
  - Use version 2.19 of the opentron API
  - Use an 8 channel pipette.
  - You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
  - Use the same set of pipette tips for the entire dilution. So you only have to pick up tips once and drop them once over the course of the entire protocol.


Use a combination of ChatGPT and the opentron API documentation to write a protocol that will perform this serial dilution on an opentron Flex using version 2.19 of the API. The newest ChatGPT should be somewhat familiar with the opentron API and should be able to write opentron python code. However, you may want to give it an example script to give hints about the specific structure and syntax you are looking to generate. You may also have to revise your script by hand using a combination of ChatGPT and the API documentation.

The protocol must simulate succesfully.

⭐ Write your answer in the code cell below and use the simulation cell below that to try the protocol

In [ ]:
from opentrons.simulate import get_protocol_api

# Create a ProtocolContext for the Flex robot
protocol = get_protocol_api(
    version="2.19", robot_type="Flex"  # API version 2.19  # Flex robot
)
print("Protocol API version:", protocol.api_version)


# ⭐ Load labware
plate =
tiprack =
reservoir =
trash =

# ⭐ Load pipette
pipette =

# ⭐ Variables
dilution_factor =
num_of_dilutions =
sample_volume =
diluent_volume =

# Pickup the pipette tips
pipette.pick_up_tip()

# Add diluent to columns 2 through 7 (270 µL in each well across all rows)
for col in range(2, num_of_dilutions + 2):
    # ⭐

# Perform serial dilution across all rows in columns 1 to 6
for col in range(1, num_of_dilutions + 1):
    # ⭐ Transfer sample from current column to the next


    # ⭐ Mix to ensure proper dilution


# ⭐ Optional: Dispose of remaining sample from the last column to avoid overflow


# Dispose of the tips
pipette.drop_tip()

---

## Coding Agents with Gemini CLI

In previous exercises you have worked with Gemini CLI and even built your own MCP tools. Each tool was designed for a specific purpose — for example, one tool to move a pipette, another to measure liquid, etc. This time, we take a different approach: instead of many small tools, you will get **one powerful coding agent**.

A coding agent is an AI assistant that can write and run code step by step while remembering the current state. This means it can share the same Python context across multiple calls. For example, if you define a variable or create a function in one step, the agent can use it again later without redefining it. This is very different from traditional one-shot tools, which start from scratch each time.

### One Tool, Many Tasks

Normally, MCP tools are built for a single action. But with a coding agent, you only need **one tool**:

```python
@mcp.tool
def run_python(code):
    exec(code, {"pipette": pipette})
```

Here, the tool accepts any Python code as input and executes it. Because the execution environment has access to objects like `pipette`, you can write different code snippets to control simulated labware — from aspirating liquid, to dispensing, to automating whole protocols. The same tool can therefore handle a wide range of tasks.


### The `exec` Function in Python

The key to making this possible is Python’s built-in `exec` function.

`exec` takes a string of Python code and runs it as if you had typed it directly into the program.

For example:

In [35]:
code = """
x = 5
print(x * 2)
"""

exec(code)

10


You can also give `exec` a **context** (a dictionary of variables and objects) where the code will run:

In [ ]:
x = 5

code = "print(x * 2)"

exec(code, {"x": x})

10


This means any code you pass in can directly use objects without having to create it again. In your case, the context includes the labware and intruments you have already set up, like the `plate`, `tiprack`, `pipette` or `trash`.

For example:

In [ ]:
protocol = get_protocol_api(version="2.19", robot_type="Flex")

plate = protocol.load_labware(
    load_name="corning_96_wellplate_360ul_flat", location="D1"
)
tiprack_1 = protocol.load_labware(
    load_name="opentrons_flex_96_tiprack_200ul", location="D2"
)
trash = protocol.load_trash_bin(location="D3")

pipette = protocol.load_instrument(
    instrument_name="flex_1channel_1000",
    mount="left",
    tip_racks=[tiprack_1],
)


def run_python(code):
    exec(
        # Execute the provided Python code
        code,
        # Provide access to the protocol objects
        {
            "plate": plate,
            "tiprack_1": tiprack_1,
            "trash": trash,
            "pipette": pipette,
        },
    )

# Example usage: run a simple pipetting command
code = """
pipette.pick_up_tip()
print("Picked up tip")
pipette.aspirate(10, plate["A1"])
print("Aspirated 10uL from A1")
pipette.dispense(10, plate["A2"])
print("Dispensed 10uL to A2")
pipette.drop_tip()
print("Dropped tip")
"""

run_python(code)

/Users/nilsm/.opentrons/robot_settings.json not found. Loading defaults
Belt calibration not found.


Picked up tip
Aspirated 10uL from A1
Dispensed 10uL to A2
Dropped tip


### Why Coding Agents?

With this approach, you no longer need to design a separate MCP tool for every single lab action. Instead:

* The coding agent uses the `run_python` tool as a **universal interface**.
* You can focus on **how to communicate with the agent**: writing clear instructions (in `Gemini.md`) and guiding it step by step to complete lab automation tasks.
* The agent can remember and build on previous steps, which makes it powerful for scripting longer workflows.


👉 In this lab, your task will be to design good instructions for the agent and practice telling it what code to run in order to automate robotic lab protocols.


### Hypha to connect your MCP tool to Gemini CLI


# TODO: add a brief explaination of hypha rpc

---

Full implementation of the code interpreter below:

In [ ]:
import ast
import io
import traceback
from contextlib import redirect_stderr, redirect_stdout
from typing import Any, Dict, Optional

from hypha_rpc import connect_to_server
from hypha_rpc.utils.schema import schema_function
from pydantic import Field

# Global variable to store lab objects
LAB_OBJECTS = {}

@schema_function
def run_python(
    code: str = Field(..., description="Python source code to execute."),
    context: Optional[Dict[str, Any]] = Field(
        default=None,
        description="Optional dictionary of variables to preload into the execution environment.",
    ),
) -> str:
    """
    Execute Python code and return combined stdout, stderr, result, or error as a single string.
    Useful for print-style outputs where we want to avoid JSON encoding issues.
    """
    # Print the code being executed
    print("Executing code:")
    print("```python")
    print(code)
    print("```")
    print("\nOutput:")
    
    context = {} if context is None else dict(context)

    # Add lab objects to context
    if LAB_OBJECTS is not None:
        context.update(LAB_OBJECTS)

    try:
        tree = ast.parse(code, mode="exec")
    except SyntaxError as e:
        error_msg = f"SyntaxError: {e}"
        print(error_msg)
        return error_msg

    # Detect last expression for REPL-style return
    capture_result = False
    if tree.body and isinstance(tree.body[-1], ast.Expr):
        capture_result = True
        last_expr = tree.body[-1].value
        tree.body[-1] = ast.Assign(
            targets=[ast.Name(id="_ai_result", ctx=ast.Store())],
            value=last_expr,
        )
        ast.fix_missing_locations(tree)

    code_obj = compile(tree, filename="<agent-code>", mode="exec")

    f_stdout, f_stderr = io.StringIO(), io.StringIO()
    try:
        with redirect_stdout(f_stdout), redirect_stderr(f_stderr):
            exec(code_obj, context)
        result = context.get("_ai_result") if capture_result else None
        out = f_stdout.getvalue()
        err = f_stderr.getvalue()
        
        # Build the complete output string
        output = ""
        if out:
            output += out
        if err:
            output += f"Stderr:\n{err}"
        if result is not None:
            output += f"Result: {result!r}"
            
        # Print the output
        print(output)
        return output
    except Exception as e:
        error_msg = f"Error: {type(e).__name__}\n{traceback.format_exc()}"
        print(error_msg)
        return error_msg

async def register_mcp_service(lab_objects: Dict[str, Any]):
    """
    Register the Python code execution service with Hypha.
    """
    global LAB_OBJECTS
    LAB_OBJECTS = lab_objects
    
    # Register the service with Hypha
    server = await connect_to_server({"server_url": "https://hypha.aicell.io"})
    svc = await server.register_service(
        {
            "id": "python-interpreter",
            "name": "Python Interpreter",
            "description": "Execute Python code with optional context; captures stdout, stderr, and last expression result.",
            "config": {
                # Make the service public so it can be used by anyone
                "visibility": "public"
            },
            # Register the function directly (not wrapped with partial)
            "run_python": run_python,
        }
    )
    
    print("========================================\n")
    print("Copy this MCP service URL to your Gemini `settings.json`:\n")
    print(f"https://hypha.aicell.io/{server.config.workspace}/mcp/{svc.id.split('/')[1]}/mcp\n")
    print("========================================\n")

    print("You can test the service function using this url:")
    print(f"https://hypha.aicell.io/{server.config.workspace}/services/{svc.id.split('/')[1]}/run_python?code=print('hi')\n")
    print("Note: Make sure to copy the full URL above!")

    # This will keep the service running forever
    await server.serve()

# Automating a real-life protocol

Now we're going to explore how we can use the combination of lab robotics and LLMs to (partly) reproduce a real-world protocol.

Our protocol will come from this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3185615/) testing the effects of various antifungal compounds on the formation of yeast biofilms.

We will see how we can speed the process of experimentation using the tools we've been learning about.

Watch the entire protocol [video](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3185615/bin/jove-44-2287-pmcvs_normal.mp4) to understand the purpose of the protocol and the steps involved.

🌞 <font color='orange'>**Exercise**:</font>

Say you've performed the protocol by hand up until the point of setting up the plate with yeast for biofilm formation (2:49 in the video). You now have your yeast culture prepared and a 96-well plate ready to use.

Choose the lab equipment you'll have to use from the opentron hardware/labware lists and use appropriate prompts to ChatGPT to create a script that will perform the protocol steps in the video outlined from 2:49 to 3:06 using an opentron Flex. You can make the following assumptions:

- You have prepared 80mL of diluted yeast sample and have transferred it to an appropriate reservoir.
- At the end of this script run, you'll lid the plate and parafilm it by hand.
- Use version 2.19 of the opentron API.
- Use an 8 channel pipette.
- You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
- Use the same set of pipette tips for the entire dilution. So you only have to pick up tips once and drop them once over the course of the entire protocol.

Simulate the protocol and make sure it runs correctly.

In [ ]:
from opentrons.simulate import get_protocol_api

# Setup Opentrons protocol and labware/instrument objects
protocol = get_protocol_api(version="2.19", robot_type="Flex")

# ⭐ Load labware
plate =
tiprack_1 =
trash =
# Define other labware as needed

# ⭐ Load instruments
pipette =
# Define other instruments as needed

lab_objects = {
    "protocol": protocol,
    "plate": plate,
    "tiprack_1": tiprack_1,
    "trash": trash,
    "pipette": pipette,
    # ⭐ Add other labware/instruments here
}

await register_mcp_service(lab_objects)

* Write a GEMINI.md file
* Add MCP tool to Gemini settings.json
* Start Gemini CLI with the MCP tool

### Work on  VS Code Tunnel

To work on your MCP tools, we recommend to switch to VS Code for better writing and debugging of your code. To still use Google Colab's free computing resources while enjoying the benefits of VS Code, you can install the VS Code Tunnel. This creates a secure connection between your Colab session and your local VS Code application, allowing you to edit files and run code directly within VS Code, while the processing happens in your Colab notebook. In addition, it will allow you to continue accessing all packages you installed in the current notebook.

To install the VS Code Tunnel, click the **Terminal** button in the bottom of this page to open a terminal window.

Then run the command below in the terminal to install the VS Code Tunnel:
```
curl -Lk 'https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64' --output vscode_cli.tar.gz
tar -xf vscode_cli.tar.gz
```

### Start VS Code Tunnel

After downloading the VS Code CLI, in the terminal, type:
```
./code tunnel
```

Then follow the instructions, use your arrow keys to select `❯ GitHub Account`, then you will see something like:

> To grant access to the server, please log into https://github.com/login/device and use code B6BB-23AA

You should now copy the url and open in a new browser tab, then copy the device code to login, you will need to approve the access to your github account then return to this terminal.

Then you will see something like, and type e.g. `colab` for the name to identify this machine:
> ? What would you like to call this machine? (30d8d79434a3) › colab

After that you should see:
> Open this link in your browser https://vscode.dev/tunnel/colab/content

Now visit the link and you should get the vscode session where you will work on.

> Switch to the Module6 Folder
Open a terminal in VS Code (View -> Terminal). Move working directory to folder on Google Drive and use this folder as your workspace. This is a crucial step.

```
cd /content/drive/MyDrive/DDLS-Course/Module6/
code .
```

🌞 <font color='orange'>**Exercise 2(Optional)**:</font>

For running this task, you will need to terminate last running cell first.

Now say you've continued the protocol, performing the incubation and washes by hand. You have prepared a stock solution of 1,024 $\mu$g/mL fluconazole and a stock solution of buffered RPMI 1640 medium.

Choose the lab equipment you'll have to use from the opentron hardware/labware lists and use appropriate prompts to ChatGPT to create a script that will perform the protocol steps for Antifungal Susceptibility Testing of Biofilms (3:57 to 5:04). You can make the following assumptions:

- You have prepared 80mL of fluconazole and 80mL of media and have transferred them to appropriate reservoirs.
- At the end of this script run, you'll lid the plate and parafilm it by hand.
- Use version 2.19 of the opentron API.
- Use an 8 channel pipette.
- You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
- Use a *different* set of pipette tips for (1) dispensing the antifungal (2) dispensing the medium (3) performing the dilutions. So you should use a total of three set of tips (pick up / drop off operations) over the course of the protocol.

Simulate the protocol and make sure it runs correctly.

In [ ]:
from opentrons.simulate import get_protocol_api

# Setup Opentrons protocol and labware/instrument objects
protocol = get_protocol_api(version="2.19", robot_type="Flex")

# ⭐ Load labware
plate =
tiprack_1 =
trash =
# Define other labware as needed

# ⭐ Load instruments
pipette =
# Define other instruments as needed

lab_objects = {
    "protocol": protocol,
    "plate": plate,
    "tiprack_1": tiprack_1,
    "trash": trash,
    "pipette": pipette,
    # ⭐ Add other labware/instruments here
}

await register_mcp_service(lab_objects)

🌞 <font color='orange'>**Discussion Exercises**:</font>

1. What were the benefits of using ChatGPT for writing these protocols? What was it able to do succesfully and what did you have to do by hand?

2. Look at the entire protocol from the paper [video](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3185615/bin/jove-44-2287-pmcvs_normal.mp4) again. What additional portions do you think you would be able to automate on the opentron? What parts could you automate with more advanced robotic hardware? What parts (if any) are do you think can't be reliably done using robotics?

3. Looking at the output of your simulations, can you propose a way in which you could have LLMs write protocols then use the simulation output to revise the protocol accordingly?

---
⭐ Double click to write down your answers here


```
Answer:

1.

2.

3.

```
---